In [ ]:
%env CUDA_LAUNCH_BLOCKING=1

In [ ]:
from experiments.attn.victory import *

In [ ]:
run()

In [ ]:
D = 32
B = 8*1024
T = 10000
device = 'cuda'

boards = arrdict.from_dicts(load_boards()).to(device)
worlds = Hex(boards.worlds)

n_boards, boardsize, _ = worlds.board.shape
model = common.AttnModel(common.PosActions, boardsize, D).to(device)

opt = torch.optim.Adam(model.parameters(), lr=1e-2)



In [ ]:
worlds.display(0)

In [ ]:
worlds.obs.sum(-1).reshape(worlds.n_envs, -1).gather(1, boards.actions[:, None]).min()